In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize

# --------------------------------------------------------
#  Case 1: Pure Linear Model
# --------------------------------------------------------
def fit_linear_model(X, y):
    """
    Fits a standard linear regression using sklearn.
    Returns a dictionary with 'intercept' and 'weights'.
    """
    reg = LinearRegression(fit_intercept=True)
    reg.fit(X, y)
    intercept = reg.intercept_
    coefs = reg.coef_  # array of shape (n,)
    
    return {'intercept': intercept, 'weights': coefs}

def predict_linear(X_new, linear_params):
    """
    Predicts y_hat for new inputs X_new under the linear model.
    linear_params: dict with 'intercept' and 'weights'
    """
    intercept = linear_params['intercept']
    weights = linear_params['weights']  # shape (n,)
    return intercept + X_new @ weights


# --------------------------------------------------------
#  Case 2: Piecewise-Linear with Threshold
# --------------------------------------------------------
def piecewise_transform(x, alpha1, alpha2):
    """ Single-feature piecewise-linear transform. """
    if x < alpha1:
        return 0.0
    elif x >= alpha2:
        return 1.0
    else:
        return (x - alpha1) / (alpha2 - alpha1)

def model_piecewise(params, X):
    """
    Returns predicted values y_hat under the piecewise-threshold model.
    params layout:
      [w0, w1..w_n, p1_1, p1_2, p2_1, p2_2, ..., pn_1, pn_2]
    with reparameterization for alpha2_j = alpha1_j + exp(...)
    """
    m, n = X.shape
    
    # Parse out w0, w[1..n]
    w0 = params[0]
    w = params[1 : n+1]  # shape (n,)
    
    # Parse out alpha1, alpha2 pairs
    alphas = []
    for j in range(n):
        base_index = n + 1 + 2*j
        alpha1 = params[base_index]
        alpha2 = alpha1 + np.exp(params[base_index + 1])
        alphas.append((alpha1, alpha2))
    
    y_hat = np.zeros(m)
    for i in range(m):
        pred_i = w0
        for j in range(n):
            alpha1, alpha2 = alphas[j]
            pred_i += w[j] * piecewise_transform(X[i, j], alpha1, alpha2)
        y_hat[i] = pred_i
    
    return y_hat

def objective_piecewise(params, X, y):
    y_hat = model_piecewise(params, X)
    return np.sum((y - y_hat)**2)

def fit_piecewise_model(X, y, max_iter=1000, random_seed=42):
    """
    Fits a piecewise-linear threshold model for every feature.
    """
    np.random.seed(random_seed)
    m, n = X.shape
    
    # total parameters = (n+1) + 2*n = 3n+1
    init = np.zeros(3*n + 1)
    
    # Initialize intercept with mean(y)
    init[0] = np.mean(y)
    
    # Initialize w_j
    init[1 : n+1] = 0.01 * np.random.randn(n)
    
    # Initialize alpha1, alpha2 for each feature
    # reparameterize alpha2 = alpha1 + exp(...)
    for j in range(n):
        base_index = n + 1 + 2*j
        col_j = X[:, j]
        q25 = np.percentile(col_j, 25)
        q75 = np.percentile(col_j, 75)
        alpha1_guess = q25
        alpha2_guess = max(q75, alpha1_guess + 1e-2)
        init[base_index] = alpha1_guess
        init[base_index + 1] = np.log(alpha2_guess - alpha1_guess)
    
    res = minimize(
        fun=objective_piecewise,
        x0=init,
        args=(X, y),
        method='L-BFGS-B',
        options={'maxiter': max_iter}
    )
    
    if not res.success:
        print("Warning: Piecewise optimization did not converge.")
    
    # Extract final
    final = res.x
    w0 = final[0]
    w = final[1 : n+1]
    alpha_pairs = []
    for j in range(n):
        base_index = n + 1 + 2*j
        a1 = final[base_index]
        a2 = a1 + np.exp(final[base_index + 1])
        alpha_pairs.append((a1, a2))
    
    return {
        'intercept': w0,
        'weights': w,
        'alphas': alpha_pairs,
        'objective_value': res.fun
    }

def predict_piecewise(X_new, piecewise_params):
    """
    Predict y_hat for new data X_new using the learned piecewise model.
    piecewise_params: dict with 'intercept', 'weights', 'alphas'
    """
    w0 = piecewise_params['intercept']
    w = piecewise_params['weights']  # shape (n,)
    alpha_pairs = piecewise_params['alphas']  # list of (alpha1, alpha2)
    
    m, n = X_new.shape
    y_hat = np.zeros(m)
    for i in range(m):
        pred_i = w0
        for j in range(n):
            alpha1, alpha2 = alpha_pairs[j]
            pred_i += w[j] * piecewise_transform(X_new[i, j], alpha1, alpha2)
        y_hat[i] = pred_i
    return y_hat


# --------------------------------------------------------
#  Case 3: Logistic (Sigmoid) for each feature
# --------------------------------------------------------
def logistic_transform(x, a, b):
    """
    Logistic function for a single feature:
    sigma(x; a, b) = 1 / (1 + exp(-b(x - a)))
    """
    return 1.0 / (1.0 + np.exp(-b * (x - a)))

def model_logistic(params, X):
    """
    Returns predicted y_hat for logistic transformations on each feature.
    params layout:
      [w0, w1..w_n, a1, b1, a2, b2, ..., an, bn]
    => length = (n+1) + 2n = 3n+1
    """
    m, n = X.shape
    w0 = params[0]
    w = params[1 : n+1]
    
    ab_pairs = []
    for j in range(n):
        base_index = n + 1 + 2*j
        a_j = params[base_index]
        b_j = params[base_index + 1]
        ab_pairs.append((a_j, b_j))
    
    y_hat = np.zeros(m)
    for i in range(m):
        pred_i = w0
        for j in range(n):
            a_j, b_j = ab_pairs[j]
            pred_i += w[j] * logistic_transform(X[i, j], a_j, b_j)
        y_hat[i] = pred_i
    return y_hat

def objective_logistic(params, X, y):
    y_hat = model_logistic(params, X)
    return np.sum((y - y_hat)**2)

def fit_logistic_model(X, y, max_iter=1000, random_seed=123):
    """
    Fits a model:
        y = w0 + sum_j w_j * logistic(x_j; a_j, b_j)
    for each feature j.
    """
    np.random.seed(random_seed)
    m, n = X.shape
    
    # total params = (n+1) + 2n = 3n+1
    init = np.zeros(3*n + 1)
    
    # Initialize intercept
    init[0] = np.mean(y)
    
    # Initialize w_j
    init[1 : n+1] = 0.01 * np.random.randn(n)
    
    # Initialize (a_j, b_j)
    for j in range(n):
        base_index = n + 1 + 2*j
        median_j = np.median(X[:, j])
        init[base_index] = median_j  # location
        init[base_index + 1] = 0.01 * np.random.randn()  # small random slope
    
    res = minimize(
        fun=objective_logistic,
        x0=init,
        args=(X, y),
        method='L-BFGS-B',
        options={'maxiter': max_iter}
    )
    
    if not res.success:
        print("Warning: Logistic optimization did not converge.")
    
    final = res.x
    w0 = final[0]
    w = final[1 : n+1]
    ab_pairs = []
    for j in range(n):
        base_index = n + 1 + 2*j
        a_j = final[base_index]
        b_j = final[base_index + 1]
        ab_pairs.append((a_j, b_j))
    
    return {
        'intercept': w0,
        'weights': w,
        'ab_pairs': ab_pairs,
        'objective_value': res.fun
    }

def predict_logistic(X_new, logistic_params):
    """
    Predict y_hat for new data X_new using the learned logistic model.
    logistic_params: dict with 'intercept', 'weights', 'ab_pairs'
    """
    w0 = logistic_params['intercept']
    w = logistic_params['weights']
    ab_pairs = logistic_params['ab_pairs']
    
    m, n = X_new.shape
    y_hat = np.zeros(m)
    for i in range(m):
        pred_i = w0
        for j in range(n):
            a_j, b_j = ab_pairs[j]
            pred_i += w[j] * logistic_transform(X_new[i, j], a_j, b_j)
        y_hat[i] = pred_i
    return y_hat




In [ ]:

# --------------------------------------------------------
#  Visualization of learned parameters
# --------------------------------------------------------
def plot_linear_params(linear_params, feature_names=None):
    """
    Show intercept and weights as a bar chart for the linear model.
    linear_params: dict with 'intercept', 'weights'
    feature_names: optional list of names for each feature.
    """
    intercept = linear_params['intercept']
    weights = linear_params['weights']
    n = len(weights)
    
    if feature_names is None:
        feature_names = [f"x_{j+1}" for j in range(n)]
    
    # We'll put intercept as the first bar, then the rest
    labels = ["Intercept"] + feature_names
    values = [intercept] + list(weights)
    
    plt.figure(figsize=(8,4))
    bars = plt.bar(labels, values, color='skyblue')
    plt.title("Linear Model Parameters")
    plt.ylabel("Parameter Value")
    plt.xticks(rotation=45, ha='right')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}",
                 ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()

def plot_piecewise_params(piecewise_params, feature_names=None):
    """
    Visualize intercept, weights, and threshold pairs for the piecewise model.
    piecewise_params: dict with 'intercept', 'weights', 'alphas'
    """
    intercept = piecewise_params['intercept']
    weights = piecewise_params['weights']
    alpha_pairs = piecewise_params['alphas']
    n = len(weights)
    
    if feature_names is None:
        feature_names = [f"x_{j+1}" for j in range(n)]
    
    # 1) Bar chart for intercept + weights
    labels = ["Intercept"] + feature_names
    values = [intercept] + list(weights)
    
    plt.figure(figsize=(8,4))
    bars = plt.bar(labels, values, color='lightgreen')
    plt.title("Piecewise Model: Intercept and Weights")
    plt.ylabel("Parameter Value")
    plt.xticks(rotation=45, ha='right')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}",
                 ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()
    
    # 2) Bar chart for threshold alphas (alpha1, alpha2)
    # Flatten them: alpha1_1, alpha2_1, alpha1_2, alpha2_2, ...
    alpha_labels = []
    alpha_vals = []
    for j, (a1, a2) in enumerate(alpha_pairs, start=1):
        alpha_labels.append(f"alpha1({feature_names[j-1]})")
        alpha_vals.append(a1)
        alpha_labels.append(f"alpha2({feature_names[j-1]})")
        alpha_vals.append(a2)
    
    plt.figure(figsize=(8,4))
    bars = plt.bar(alpha_labels, alpha_vals, color='orange')
    plt.title("Piecewise Model: Threshold Parameters")
    plt.ylabel("Threshold Value")
    plt.xticks(rotation=45, ha='right')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}",
                 ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()

def plot_logistic_params(logistic_params, feature_names=None):
    """
    Visualize intercept, weights, and logistic parameters (a_j, b_j).
    logistic_params: dict with 'intercept', 'weights', 'ab_pairs'
    """
    intercept = logistic_params['intercept']
    weights = logistic_params['weights']
    ab_pairs = logistic_params['ab_pairs']
    n = len(weights)
    
    if feature_names is None:
        feature_names = [f"x_{j+1}" for j in range(n)]
    
    # 1) Bar chart for intercept + weights
    labels = ["Intercept"] + feature_names
    values = [intercept] + list(weights)
    
    plt.figure(figsize=(8,4))
    bars = plt.bar(labels, values, color='pink')
    plt.title("Logistic Model: Intercept and Weights")
    plt.ylabel("Parameter Value")
    plt.xticks(rotation=45, ha='right')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}",
                 ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()
    
    # 2) Bar chart for logistic parameters (a_j, b_j)
    ab_labels = []
    ab_vals = []
    for j, (a_j, b_j) in enumerate(ab_pairs, start=1):
        ab_labels.append(f"a({feature_names[j-1]})")
        ab_vals.append(a_j)
        ab_labels.append(f"b({feature_names[j-1]})")
        ab_vals.append(b_j)
    
    plt.figure(figsize=(8,4))
    bars = plt.bar(ab_labels, ab_vals, color='salmon')
    plt.title("Logistic Model: (a_j, b_j) Parameters")
    plt.ylabel("Value")
    plt.xticks(rotation=45, ha='right')
    for bar in bars:
        yval = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2, yval, f"{yval:.2f}",
                 ha="center", va="bottom", fontsize=9)
    plt.tight_layout()
    plt.show()


# --------------------------------------------------------
# Example usage / test
# --------------------------------------------------------
if __name__ == "__main__":
    # Generate synthetic data
    np.random.seed(42)
    m = 100
    n = 2
    X = np.random.randn(m, n)
    
    # Construct some y using a hidden combination:
    #   y = 3 + 2 * piecewise_transform(x1, -0.5, 0.5) + 4 * logistic_transform(x2, a=0, b=5) + noise
    def true_piecewise(x):
        if x < -0.5:
            return 0.0
        elif x > 0.5:
            return 1.0
        else:
            return (x + 0.5)/1.0
    
    def true_logistic(x):
        return 1.0 / (1.0 + np.exp(-5 * (x - 0.0)))
    
    noise = 0.05 * np.random.randn(m)
    y = []
    for i in range(m):
        x1, x2 = X[i]
        val = 3.0 + 2.0*true_piecewise(x1) + 4.0*true_logistic(x2)
        val += noise[i]
        y.append(val)
    y = np.array(y)
    
    # Fit each model
    lin_model = fit_linear_model(X, y)
    piecewise_model = fit_piecewise_model(X, y)
    logistic_model = fit_logistic_model(X, y)
    
    # Visualize parameters
    feature_names = ["Feature1", "Feature2"]
    plot_linear_params(lin_model, feature_names)
    plot_piecewise_params(piecewise_model, feature_names)
    plot_logistic_params(logistic_model, feature_names)
    
    # Predict on new data points
    X_new = np.array([
        [-1.0, -0.2],
        [0.0, 0.0],
        [0.4, 1.2]
    ])
    print("\nX_new =\n", X_new)

    y_hat_lin = predict_linear(X_new, lin_model)
    y_hat_pwl = predict_piecewise(X_new, piecewise_model)
    y_hat_log = predict_logistic(X_new, logistic_model)

    print("\nPredictions, linear model =", y_hat_lin)
    print("Predictions, piecewise model =", y_hat_pwl)
    print("Predictions, logistic model =", y_hat_log)